# Import module

The link to get [ImageAnalysis3](https://github.com/zhengpuas47/ImageAnalysis3) 

Or from the Zhuang lab archived [source_tools](https://github.com/ZhuangLab/Chromatin_Analysis_2020_cell/tree/master/sequential_tracing/source)

## ImageAnalysis3 and basic modules

In [21]:
%run "C:\Users\shiwei\Documents\ImageAnalysis3\required_files\Startup_py3.py"
sys.path.append(r"C:\Users\shiwei\Documents")

import ImageAnalysis3 as ia
from ImageAnalysis3 import *
from ImageAnalysis3.classes import _allowed_kwds

import h5py
import ast
import pandas as pd

print(os.getpid())

9128


## Chromatin_analysis_tools etc

See **functions** in the repository for [AnalysisTool_Chromatin](../../README.md)

In [22]:
# Chromatin_analysis_tools (ATC)
# Get path for the py containing functions
import os
import sys
import importlib
module_path =r'C:\Users\shiwei\Documents\AnalysisTool_Chromatin'
if module_path not in sys.path:
    sys.path.append(module_path)
    
# import relevant modules
import gene_selection 
importlib.reload(gene_selection)
import gene_to_loci
importlib.reload(gene_to_loci)
import gene_activity
importlib.reload(gene_activity)
import loci_1d_features
importlib.reload(loci_1d_features)  

import atac_to_loci
importlib.reload(atac_to_loci)

<module 'atac_to_loci' from 'C:\\Users\\shiwei\\Documents\\AnalysisTool_Chromatin\\atac_to_loci.py'>

# Define folders

In [23]:
# main folder for postanalysis
postanalysis_folder = r'L:\Shiwei\postanalysis_2024\v0'
# input files for postanalysis
input_folder = os.path.join(postanalysis_folder, 'resources_from_preprocess')

# output file to be generated
output_main_folder = os.path.join(postanalysis_folder, 'radial_position')

output_analysis_folder = os.path.join(output_main_folder, 'analysis')
output_figure_folder = os.path.join(output_main_folder, 'figures')

# make new folder if needed
make_output_folder = True

if make_output_folder and not os.path.exists(output_analysis_folder):
    os.makedirs(output_analysis_folder)
    print(f'Generating analysis folder: {output_analysis_folder}.')
elif os.path.exists(output_analysis_folder):
    print(f'Use existing analysis folder: {output_analysis_folder}.')
    
if make_output_folder and not os.path.exists(output_figure_folder):
    os.makedirs(output_figure_folder)
    print(f'Generating figure folder: {output_figure_folder}.')
elif os.path.exists(output_figure_folder):
    print(f'Use existing figure folder: {output_figure_folder}.')

Use existing analysis folder: L:\Shiwei\postanalysis_2024\v0\radial_position\analysis.
Use existing figure folder: L:\Shiwei\postanalysis_2024\v0\radial_position\figures.


# Plotting parameters

In [24]:
%matplotlib inline
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
import matplotlib.pyplot as plt
plt.rc('font', family='serif')
plt.rc('font', serif='Arial')

from ImageAnalysis3.figure_tools import _double_col_width, _single_col_width, _font_size, _ticklabel_size,_ticklabel_width

import seaborn as sns
sns.set_context("paper", rc={"font.size":_font_size,"axes.titlesize":_font_size+1,"axes.labelsize":_font_size})  

In [25]:
# Other required plotting parameters
_dpi = 300
_font_size = 7
_page_width = 5.5


## cell type color-codes

In [26]:
# cell labels from RNA-MERFISH and celltype prediction
selected_cell_labels = ['L2/3 IT','L4/5 IT','L5 IT','L6 IT','L5 ET','L5/6 NP','L6 CT','L6b',
                           'Sst','Pvalb','Lamp5','Sncg','Vip',
                           'Astro','Oligo','OPC','Micro','Endo','VLMC','SMC','Peri', 
                           #'other',
                          ]
# cell palette from RNA-MERFISH UMAP and stats
celltype_palette = {'Astro':'lightcoral', 
                    'Endo':'skyblue', 
                    'L2/3 IT':'gold', 
                    'L4/5 IT':'darkorange', 
                    'L5 ET':'mediumseagreen', 
                    'L5 IT':'aqua',
                    'L5/6 NP':'darkgreen',
                    'L6 CT':'brown',
                    'L6 IT':'magenta',
                    'L6b':'blue', 
                    'Lamp5':'orange', 
                    'Micro':'peachpuff',
                    'OPC':'thistle', 
                    'Oligo':'darkviolet',
                    'Peri':'sandybrown',
                    'Pvalb':'springgreen',
                    'SMC':'rosybrown',
                    'Sncg':'darkkhaki',
                    'Sst':'steelblue', 
                    'VLMC':'saddlebrown', 
                    'Vip':'red',
                    'other':'slategray'}


In [27]:
# this is the plotting order noted based on the snRNA transcriptional acitivty if needed
sorted_cellplot_order_byRNA = ['Micro', 'Oligo', 'Endo', 'OPC', 'Astro', 'Vip', 'Lamp5',
                  'L5/6 NP', 'Sst', 'Sncg', 'Pvalb', 'L4/5 IT', 'L6 CT',
                  'L6 IT', 'L6b', 'L2/3 IT', 'L5 IT', 'L5 ET']

# Load data relevant information

## load and format codebook

[merged codebook](../resources/merged_codebook.csv) as in the repository (merged for all DNA-MERFISH libraries)

In [28]:
# Load codebook 
codebook_fname = os.path.join(input_folder,'merged_codebook.csv')
codebook_df = pd.read_csv (codebook_fname, index_col=0)

# sort df by chr and chr_order
codebook_df = loci_1d_features.sort_loci_df_by_chr_order (codebook_df)
codebook_df.head()

,name,id,NDB_784,NDB_755,NDB_826,NDB_713,NDB_865,NDB_725,NDB_817,NDB_710,...,NDB_479,NDB_562,NDB_608,NDB_460,NDB_563,NDB_592,NDB_368,NDB_436,NDB_629,NDB_604
0,1:3742742-3759944,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1:6245958-6258969,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1:8740008-8759916,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1016,1:9627926-9637875,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1017,1:9799472-9811359,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [29]:
# Format the chr loci name by 
# 1. changing loci name format
# 2. extract relevant information such as id, chr, chr_order, and library etc
from gene_to_loci import loci_pos_format
loci_name_list = list(map(loci_pos_format, codebook_df['name'].tolist()))
loci_name_arr = np.array(loci_name_list)

# convert to a new dataframe and set loci name as index
codebook_df = codebook_df[['name','id','chr','chr_order','library']]
codebook_df['loci_name'] = list(loci_name_arr[:,0])
codebook_df = codebook_df.set_index ('loci_name')

codebook_df.head()

,name,id,chr,chr_order,library
loci_name,,,,,
chr1_3742742_3759944,1:3742742-3759944,1,1,0.0,CTP11
chr1_6245958_6258969,1:6245958-6258969,2,1,1.0,CTP11
chr1_8740008_8759916,1:8740008-8759916,3,1,2.0,CTP11
chr1_9627926_9637875,1:9627926-9637875,1,1,3.0,CTP13
chr1_9799472_9811359,1:9799472-9811359,2,1,4.0,CTP13


# Load single cell radial position

In [30]:
output_fname = os.path.join(output_analysis_folder, f'radial_pos_convexhull_bysubclass_v2.h5')

merged_radial_pos_df = pd.read_hdf(output_fname)

min_pts_th = 600
max_pts_th =3900

merged_radial_pos_df = merged_radial_pos_df[(merged_radial_pos_df['total_spots']>=min_pts_th)
                                            &(merged_radial_pos_df['total_spots']<max_pts_th)]
merged_radial_pos_df.head()

,subclass,total_spots,radial_position
uid,,,
259202492748634617304623818845147108919,Oligo,863,loci_name fiber_id norm...
336074706103419484501535095917410831417,Oligo,1426,loci_name fiber_id norm...
179117357717369278884189379535888871018,Oligo,1601,loci_name fiber_id norm...
92271180002801802991585284299263797402,Oligo,2034,loci_name fiber_id norm...
237111112842863990529297306931448726772,Oligo,942,loci_name fiber_id norm...


In [31]:
merged_radial_pos_df.iloc[70]['radial_position']

,loci_name,fiber_id,norm_radial_position
0,chr11_3752701_3759988,1,0.696515
1,chr11_4973207_4984540,1,0.692250
2,chr11_5400059_5411584,1,0.651500
3,chr11_6248948_6259774,1,0.728170
4,chr11_6586837_6594983,1,0.917523
...,...,...,...
1774,chrX_163750534_163758706,1,0.774136
1775,chrX_166247682_166259932,1,0.703176
1776,chrX_167157164_167167452,1,0.804446
1777,chrX_168746045_168757590,1,NaN


## process radial positoin by chromosome

In [32]:
chrom_list = [c for c in np.unique(codebook_df['chr']) if c!= 'Y']
chrom_list = [c for c in chrom_list  if c!='X']
chrom_list.sort(key = int)
chrom_list.append('X')

In [33]:
# get rp for single cell as list
def single_cell_rp_by_loci (radial_pos_df_cell, loci_name):
    rp_loci_df = radial_pos_df_cell[radial_pos_df_cell['loci_name']==loci_name]
    if len(rp_loci_df)>0:
        rp_loci = rp_loci_df['norm_radial_position'].tolist()
    else:
        rp_loci = [np.nan, np.nan]
    return rp_loci


In [34]:
_chr = '1'
_loci_name_list = codebook_df[codebook_df['chr']==_chr].index.tolist()
_loci_name_list [:5]

['chr1_3742742_3759944',
 'chr1_6245958_6258969',
 'chr1_8740008_8759916',
 'chr1_9627926_9637875',
 'chr1_9799472_9811359']

In [35]:
import tqdm

output_analysis_subfolder = os.path.join(output_analysis_folder,'radial_pos_byChr')
if not os.path.exists(output_analysis_subfolder):
    os.mkdir(output_analysis_subfolder)

for _chr in chrom_list[:]:

    radial_chr_fname = os.path.join(output_analysis_subfolder, f'Chr{_chr}_single_cell_radial_position.h5')
    _loci_name_list = codebook_df[codebook_df['chr']==_chr].index.tolist()

    if not os.path.exists(radial_chr_fname):
        print('Extracting radial position')

        sc_radial_dict = {}
        for _loci in tqdm.tqdm(_loci_name_list):
            sc_radial_dict[f'{_loci}_rp'] = []

            for _uid in merged_radial_pos_df.index.tolist():
                radial_pos_df_cell = merged_radial_pos_df.loc[_uid]['radial_position']
                rp_loci = single_cell_rp_by_loci (radial_pos_df_cell, _loci)
                sc_radial_dict[f'{_loci}_rp'].append(rp_loci)


        sc_radial_df = pd.DataFrame(sc_radial_dict)    
        sc_radial_df.index = merged_radial_pos_df.index
        sc_radial_df.to_hdf(radial_chr_fname,key='df', mode='w')

    else:
        continue


Extracting radial position


100%|████████████████████████████████████████████████████████████████████████████████| 153/153 [46:27<00:00, 18.22s/it]
C:\Users\shiwei\AppData\Local\Temp\3\ipykernel_9128\4245539016.py:27: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['chr1_3742742_3759944_rp', 'chr1_6245958_6258969_rp',
       'chr1_8740008_8759916_rp', 'chr1_9627926_9637875_rp',
       'chr1_9799472_9811359_rp', 'chr1_11247744_11257616_rp',
       'chr1_13741888_13757922_rp', 'chr1_16251322_16259969_rp',
       'chr1_18749330_18759924_rp', 'chr1_20920666_20929358_rp',
       ...
       'chr1_186665592_186673783_rp', 'chr1_187669688_187679262_rp',
       'chr1_188751447_188759965_rp', 'chr1_189040928_189050246_rp',
       'chr1_189207390_189217329_rp', 'chr1_189510684_189521211_rp',
       'chr1_189900300_189910420_rp', 'chr1_191250765_191259922_rp',
       'chr1_193105714_193114935_

Extracting radial position


100%|████████████████████████████████████████████████████████████████████████████████| 148/148 [44:50<00:00, 18.18s/it]
C:\Users\shiwei\AppData\Local\Temp\3\ipykernel_9128\4245539016.py:27: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['chr2_3156187_3166922_rp', 'chr2_3751359_3759973_rp',
       'chr2_4052881_4061655_rp', 'chr2_5181586_5192683_rp',
       'chr2_6250329_6259980_rp', 'chr2_8748765_8759968_rp',
       'chr2_11250804_11259903_rp', 'chr2_11345065_11361468_rp',
       'chr2_13751945_13759994_rp', 'chr2_14934135_14944278_rp',
       ...
       'chr2_172132642_172155589_rp', 'chr2_172424300_172436124_rp',
       'chr2_172844150_172852688_rp', 'chr2_173748299_173759982_rp',
       'chr2_178147161_178155385_rp', 'chr2_178508102_178517909_rp',
       'chr2_178695618_178705173_rp', 'chr2_178751849_178759991_rp',
       'chr2_180439020_180448010_rp

Extracting radial position


...

100%|██████████████████████████████████████████████████████████████████████████████████| 68/68 [20:35<00:00, 18.17s/it]
C:\Users\shiwei\AppData\Local\Temp\3\ipykernel_9128\4245539016.py:27: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['chr18_3740013_3759954_rp', 'chr18_6245525_6256029_rp',
       'chr18_6969332_6981478_rp', 'chr18_8751060_8759962_rp',
       'chr18_11248958_11259646_rp', 'chr18_13751864_13759962_rp',
       'chr18_15400982_15420982_rp', 'chr18_15420635_15430854_rp',
       'chr18_16249668_16260001_rp', 'chr18_18743382_18759727_rp',
       'chr18_21177743_21186774_rp', 'chr18_21252243_21259983_rp',
       'chr18_23751547_23759968_rp', 'chr18_24115969_24126881_rp',
       'chr18_24451335_24462709_rp', 'chr18_24800347_24810452_rp',
       'chr18_26246003_26259974_rp', 'chr18_28740801_28759965_rp',
       'chr18_31244964_31259982_rp', 'ch

Extracting radial position


100%|██████████████████████████████████████████████████████████████████████████████████| 59/59 [17:48<00:00, 18.10s/it]
C:\Users\shiwei\AppData\Local\Temp\3\ipykernel_9128\4245539016.py:27: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['chr19_3665068_3674589_rp', 'chr19_3746331_3759074_rp',
       'chr19_3870582_3880008_rp', 'chr19_5661977_5670182_rp',
       'chr19_6253105_6259999_rp', 'chr19_6296114_6304474_rp',
       'chr19_7132233_7140214_rp', 'chr19_8560165_8581310_rp',
       'chr19_8749899_8759800_rp', 'chr19_10712968_10721926_rp',
       'chr19_10946469_10958550_rp', 'chr19_11251370_11259820_rp',
       'chr19_12471732_12480838_rp', 'chr19_13740063_13759965_rp',
       'chr19_15798183_15810721_rp', 'chr19_16252365_16260001_rp',
       'chr19_17314559_17337782_rp', 'chr19_18750800_18759988_rp',
       'chr19_19101196_19121196_rp', 'chr19_205375

Extracting radial position


100%|██████████████████████████████████████████████████████████████████████████████████| 64/64 [18:55<00:00, 17.74s/it]
C:\Users\shiwei\AppData\Local\Temp\3\ipykernel_9128\4245539016.py:27: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['chrX_6251289_6260001_rp', 'chrX_11252408_11259994_rp',
       'chrX_13742251_13759996_rp', 'chrX_16240013_16258620_rp',
       'chrX_18740016_18759980_rp', 'chrX_21246116_21254576_rp',
       'chrX_23742166_23759800_rp', 'chrX_36247056_36259801_rp',
       'chrX_38748472_38759587_rp', 'chrX_41244470_41259864_rp',
       'chrX_43740000_43757195_rp', 'chrX_46248326_46259985_rp',
       'chrX_51241370_51256067_rp', 'chrX_53752407_53759853_rp',
       'chrX_56241726_56259819_rp', 'chrX_58747434_58759996_rp',
       'chrX_61240330_61259175_rp', 'chrX_63743031_63759849_rp',
       'chrX_66246542_66259902_rp', 'chrX_68740368_6